In [3]:
import dlib

# Initialize dlib's face detector (HOG-based) and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


In [4]:
import cv2

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)


In [5]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale (required for Dlib)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)


KeyboardInterrupt: 

In [ ]:
    for face in faces:
        # Determine the facial landmarks for the face region
        landmarks = predictor(gray, face)
        landmarks = [(p.x, p.y) for p in landmarks.parts()]


In [ ]:
def get_eye_coordinates(landmarks):
    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]
    return left_eye, right_eye

def eye_center(eye):
    x = int(np.mean([point[0] for point in eye]))
    y = int(np.mean([point[1] for point in eye]))
    return x, y


In [14]:
import cv2
import dlib
import numpy as np
import pandas as pd

# Initialize dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Function to get the eye coordinates
def get_eye_coordinates(landmarks):
    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]
    return left_eye, right_eye

# Function to calculate the center of the eye
def eye_center(eye):
    x = int(np.mean([point[0] for point in eye]))
    y = int(np.mean([point[1] for point in eye]))
    return x, y

# Initialize video capture
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

gaze_points = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) == 0:
        print("No faces detected")
        
    for face in faces:
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (255, 0, 0), 2)
        
        landmarks = predictor(gray, face)
        landmarks = [(p.x, p.y) for p in landmarks.parts()]

        # Draw landmarks
        for (x, y) in landmarks:
            cv2.circle(frame, (x, y), 1, (255, 255, 0), -1)

        left_eye, right_eye = get_eye_coordinates(landmarks)
        left_eye_center = eye_center(left_eye)
        right_eye_center = eye_center(right_eye)

        gaze_points.append(left_eye_center)
        gaze_points.append(right_eye_center)

        # Draw eyes centers
        cv2.circle(frame, left_eye_center, 3, (0, 255, 0), -1)
        cv2.circle(frame, right_eye_center, 3, (0, 255, 0), -1)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:  # ESC key to break
        break

cap.release()
cv2.destroyAllWindows()

# Check if gaze points were captured
if gaze_points:
    # Save gaze points to a CSV file
    gaze_df = pd.DataFrame(gaze_points, columns=['x', 'y'])
    gaze_df.to_csv('gaze_points.csv', index=False)
    print(f"Saved {len(gaze_points)} gaze points to gaze_points.csv")
else:
    print("No gaze points captured")

# Load an image to map gaze points
image_path = 'your_image.jpg'
image = cv2.imread(image_path)

if image is None:
    print(f"Cannot read image from {image_path}")
    exit()

# Draw gaze points on the image if there are any
if gaze_points:
    for point in gaze_points:
        cv2.circle(image, point, 5, (0, 0, 255), -1)

    cv2.imshow("Gaze Points", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No gaze points to display on the image")


Cannot open camera
Failed to capture frame
No gaze points captured
Cannot read image from your_image.jpg
No gaze points to display on the image


[ WARN:0@1103.426] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1103.426] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@1103.434] global loadsave.cpp:241 findDecoder imread_('your_image.jpg'): can't open/read file: check file path/integrity


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2

# Load gaze points from the CSV file
gaze_df = pd.read_csv('/home/boasp/Documents/Projects/eye_tracking/gaze_points.csv')

# Load the image
image = cv2.imread('your_image.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Plot the gaze points on the image
plt.figure(figsize=(10, 8))
plt.imshow(image_rgb)

# Extract gaze points
gaze_points = gaze_df.values
x = gaze_points[:, 0]
y = gaze_points[:, 1]

plt.scatter(x, y, c='red', s=10)
plt.title("Gaze Points on Image")
plt.show()


[ WARN:0@798.710] global loadsave.cpp:241 findDecoder imread_('your_image.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [9]:
import seaborn as sns
import numpy as np

# Create a blank image with the same dimensions as the original
heatmap, xedges, yedges = np.histogram2d(x, y, bins=(image.shape[1], image.shape[0]))

# Plot the heatmap
plt.figure(figsize=(10, 8))
plt.imshow(image_rgb, extent=[0, image.shape[1], image.shape[0], 0])
plt.imshow(heatmap.T, cmap='jet', alpha=0.5, extent=[0, image.shape[1], image.shape[0], 0])
plt.title("Heatmap of Gaze Points")
plt.colorbar(label='Frequency')
plt.show()


NameError: name 'x' is not defined